In [1]:
#%%
import pickle
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})
import numpy as np
from itertools import product
import seaborn as sns

### MAIN HYPERPARAMS ###
ntrees = 0
slots = 10
shifts = 6
alg_num = 1
task_num = 10
model = "dnn"
########################

#%%
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def get_fte_bte(err, single_err, ntrees):
    bte = [[] for i in range(10)]
    te = [[] for i in range(10)]
    fte = []
    
    for i in range(10):
        for j in range(i,10):
            #print(err[j][i],j,i)
            bte[i].append(err[i][i]/err[j][i])
            te[i].append(single_err[i]/err[j][i])
                
    for i in range(10):
        #print(single_err[i],err[i][i])
        fte.append(single_err[i]/err[i][i])
            
            
    return fte,bte,te

def calc_mean_bte(btes,task_num=10,reps=6):
    mean_bte = [[] for i in range(task_num)]


    for j in range(task_num):
        tmp = 0
        for i in range(reps):
            tmp += np.array(btes[i][j])
        
        tmp=tmp/reps
        mean_bte[j].extend(tmp)
            
    return mean_bte     

def calc_mean_te(tes,task_num=10,reps=6):
    mean_te = [[] for i in range(task_num)]

    for j in range(task_num):
        tmp = 0
        for i in range(reps):
            tmp += np.array(tes[i][j])
        
        tmp=tmp/reps
        mean_te[j].extend(tmp)
            
    return mean_te 

def calc_mean_fte(ftes,task_num=10,reps=6):
    fte = np.asarray(ftes)
    
    return list(np.mean(np.asarray(fte_tmp),axis=0))

def calc_mean_err(err,task_num=10,reps=6):
    mean_err = [[] for i in range(task_num)]


    for j in range(task_num):
        tmp = 0
        for i in range(reps):
            tmp += np.array(err[i][j])
        
        tmp=tmp/reps
        #print(tmp)
        mean_err[j].extend([tmp])
            
    return mean_err     

#%%
reps = slots*shifts

btes = [[] for i in range(task_num)]
ftes = [[] for i in range(task_num)]
tes = [[] for i in range(task_num)]
err_ = [[] for i in range(task_num)]

te_tmp = [[] for _ in range(reps)]
bte_tmp = [[] for _ in range(reps)]
fte_tmp = [[] for _ in range(reps)]
err_tmp = [[] for _ in range(reps)]

count = 0   
for slot in range(slots):
    for shift in range(shifts):
        filename = 'result/'+model+str(ntrees)+'_'+str(shift+1)+'_'+str(slot)+'.pickle'
        multitask_df, single_task_df = unpickle(filename)

        err = [[] for _ in range(10)]

        for ii in range(10):
            err[ii].extend(
             1 - np.array(
                 multitask_df[multitask_df['base_task']==ii+1]['accuracy']
             )
            )
        single_err = 1 - np.array(single_task_df['accuracy'])
        fte, bte, te = get_fte_bte(err,single_err,ntrees)
    
        err_ = [[] for i in range(task_num)]
        for i in range(task_num):
            for j in range(task_num-i):
                #print(err[i+j][i])
                err_[i].append(err[i+j][i])
            
        te_tmp[count].extend(te)
        bte_tmp[count].extend(bte)
        fte_tmp[count].extend(fte)
        err_tmp[count].extend(err_)
        count+=1
    
te = calc_mean_te(te_tmp,reps=reps)
bte = calc_mean_bte(bte_tmp,reps=reps)
fte = calc_mean_fte(fte_tmp,reps=reps)
error = calc_mean_err(err_tmp,reps=reps)


Bad key "text.kerning_factor" on line 4 in
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
flat_te_per_rep = [[np.mean(te_tmp[rep][i]) for i in range(len(te))] for rep in range(reps)]
mean_te_per_rep = np.mean(flat_te_per_rep, axis = 1)
min_te_per_rep = np.min(flat_te_per_rep, axis = 1)

In [3]:
print("Mean TE: ({} +- {})".format(np.mean(mean_te_per_rep), np.std(mean_te_per_rep)))
print("Min TE: ({} +- {})".format(np.mean(min_te_per_rep), np.std(min_te_per_rep)))

Mean TE: (1.2151131332308263 +- 0.014993975109554573)
Min TE: (1.094692661815922 +- 0.0315075161426492)


In [4]:
task_fte_task_10_per_rep = [fte_tmp[rep][9] for rep in range(reps)]
task_bte_task_1_per_rep = [bte_tmp[rep][0][9] for rep in range(reps)]

In [5]:
print("Mean FTE(Task 10): ({} +- {})".format(np.mean(task_fte_task_10_per_rep), np.std(task_fte_task_10_per_rep)))
print("Mean BTE(Task 1): ({} +- {})".format(np.mean(task_bte_task_1_per_rep), np.std(task_bte_task_1_per_rep)))

Mean FTE(Task 10): (1.363911173747584 +- 0.06092776038764332)
Mean BTE(Task 1): (1.183417340202393 +- 0.059798632137465835)


In [6]:
for task in range(task_num):
    final_te_of_task_per_rep = [te_tmp[rep][task][-1] for rep in range(reps)]
    print("Final TE of Task {}: ({} +- {})".format(task, np.mean(final_te_of_task_per_rep), np.std(final_te_of_task_per_rep)))

Final TE of Task 0: (1.183417340202393 +- 0.059798632137465835)
Final TE of Task 1: (1.1573109953796412 +- 0.0474808924470105)
Final TE of Task 2: (1.1693928884540874 +- 0.053910192392614256)
Final TE of Task 3: (1.2119055520937985 +- 0.04960775063565548)
Final TE of Task 4: (1.1988468581884244 +- 0.05287406679722945)
Final TE of Task 5: (1.31790836452783 +- 0.055738539734944095)
Final TE of Task 6: (1.2531010506687073 +- 0.07356446960061813)
Final TE of Task 7: (1.2020854708865125 +- 0.049530099356080125)
Final TE of Task 8: (1.247174618428836 +- 0.046167947227546656)
Final TE of Task 9: (1.363911173747584 +- 0.06092776038764332)
